In [1]:
import pandas as pd
import numpy as np 

import json
from pandas import json_normalize

import matplotlib
from matplotlib import colors
import matplotlib.pyplot as plt
import folium

from bs4 import BeautifulSoup
import geopy
import requests

In [2]:
url_mumbai='https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai'
url_delhi='https://en.wikipedia.org/wiki/List_of_neighbourhoods_of_Delhi'

In [3]:
def mumbai_neighbor(url_mumbai):
    neighbor_mumbai=[]
    result=requests.get(url_mumbai).text
    soup=BeautifulSoup(result,'html.parser')
    for i in soup.find_all('h3'):
        x=i.find('a')
        try:
            neighbor_mumbai.append(x.next_element)
        except:
            pass
    return neighbor_mumbai

In [4]:
def delhi_neighbor(urld):
    neighbor_delhi=[]
    html_data_delhi=requests.get(url_delhi).text
    soup_delhi=BeautifulSoup(html_data_delhi,'html.parser')
    for i in soup_delhi.find_all('li'):
        try:
            x=i.a   
            if '<' in str(x.next_element):
                pass
            else:
                neighbor_delhi.append(x.next_element)
        except:
            neighbor_delhi.append(list(i.descendants)[0])
    return neighbor_delhi[:119]

In [5]:
delhi_neighbors=delhi_neighbor(url_delhi)

In [6]:
def location_finder(x,state):
    address='{}, {}'.format(x['Neighborhood'],state)
    geolocator=geopy.geocoders.Nominatim(user_agent='neighod')
    g=geolocator.geocode(address)
    try:
        x.loc['Latitude']=g.latitude
        x.loc['Longitude']=g.longitude
        x.loc['Importance']=g.raw['importance']
    except:
        x.loc['Latitude']=np.nan
        x.loc['Longitude']=np.nan
        x.loc['Importance']=np.nan


In [7]:
def dataframe_creation(data_list,state):
    columns=['Neighborhood','Latitude','Longitude','Importance']
    dataframe=pd.DataFrame(columns=columns)
    dataframe['Neighborhood']=data_list
    dataframe.apply(location_finder,axis=1,args=(state,))
    return dataframe

In [8]:
delhi_dataframe=dataframe_creation(delhi_neighbors,'Delhi')
delhi_dataframe

,Neighborhood,Latitude,Longitude,Importance
0,Adarsh Nagar,28.7166,77.1704,0.708428
1,Ashok Vihar,28.6995,77.1848,0.311
2,Begum Pur,28.7255,77.0584,0.585
3,Karala,28.7351,77.0325,0.47
4,Narela,28.8426,77.0918,0.500666
...,...,...,...,...
114,Rajouri Garden,28.6451,77.1239,0.56
115,Tihar Village,28.6346,77.1071,0.373151
116,Tilak Nagar,28.6365,77.0965,0.682128
117,Vikas Nagar,28.644,77.0545,0.56


In [9]:
delhi_dataframe.to_csv('Data/delhi_dataframe.csv')

In [10]:
mumbai_neighbors=mumbai_neighbor(url_mumbai)

In [11]:
mumbai_dataframe=dataframe_creation(mumbai_neighbors,'Mumbai')

In [12]:
mumbai_dataframe

,Neighborhood,Latitude,Longitude,Importance
0,Andheri,19.1197,72.8464,0.580152
1,Mira-Bhayandar,19.2821,72.8741,0.311
2,Bandra,19.055,72.8402,0.559056
3,Borivali,19.2291,72.8574,0.58818
4,Dahisar,19.2495,72.8596,0.572959
5,Goregaon,19.1648,72.85,0.557785
6,Jogeshwari,19.1349,72.8488,0.498689
7,Juhu,19.107,72.8275,0.598549
8,Kandivali west,19.2084,72.8422,0.585
9,Kandivali east,19.2104,72.8641,0.585


In [13]:
def map_creater(dataframe,state,user_agent='map_default'):
    geolocator=geopy.geocoders.Nominatim(user_agent=user_agent)
    g=geolocator.geocode(state)
    map_created=folium.Map(location=[g.latitude,g.longitude],zoom_start=11)
    for index,row in dataframe.iterrows():
        if row.Latitude is not np.nan or row.Longitude is not np.nan:
            array=plt.cm.Spectral(row['Importance'])
            color=colors.rgb2hex(array)
            label='{},{}'.format(row['Neighborhood'],state)
            label=folium.Popup(label,parse_html=False)
            folium.CircleMarker(location=[row['Latitude'],row['Longitude']],
                                radius=10,
                                weight=2,
                                color='black',
                                fill=True,
                                fill_color=color,
                                fill_opacity =0.8,
                                popup=label).add_to(map_created)
    return map_created

In [14]:
map_creater(delhi_dataframe,'Delhi')


In [15]:
map_creater(mumbai_dataframe,'Mumbai')

In [16]:
def top_10(dataframe):
    dataframe['Importance']=dataframe['Importance'].astype('float')
    dataframe=dataframe.sort_values(by='Importance',ascending=False)
    dataframe=dataframe.round({'Importance':2})
    return dataframe[:10] 

In [17]:
top_delhi=top_10(delhi_dataframe)
top_mumbai=top_10(mumbai_dataframe)

In [18]:
def get_credentials():
    Client_Id='30YJZ4SVOKWOSS5A3B5XVEDPGKRA0GLAUIU4GMKZ1K5KLS1C'
    Client_Secret='IE1AV21QGBW1AJMA4WUKPUVMIOPWIXPENYVKLTRYAC5VSZEM'
    Version='20200502'
    Limit=500
    Radius=50
    return [Client_Id,Client_Secret,Version,Limit,Radius]

In [19]:
def get_places(dataframe):
    new_data=pd.DataFrame(columns=['Name','Categories','Latitude','Longitude'])
    for index,row in dataframe.iterrows():
        url_foursquare='https://api.foursquare.com/v2/venues/explore?&client_id={0[0]}&client_secret={0[1]}&v={0[2]}&ll={1},{2}&radius={0[4]}&limit={0[3]}'.format(get_credentials(),row['Latitude'],row['Longitude'])
        result=requests.get(url_foursquare).json()
        #print(result['response']['groups'],end='\n\n\n')

        try:
            data=json_normalize(result['response']['groups'][0]['items'])
            new_data.iloc[index,'Name']=data['venue.name']
            new_data.iloc[index,'Categories']=data['venue.categories']
            new_data.iloc[index,'Latitude']=data['venue.location.lat']
            new_data.iloc[index,'Longitude']=data['venue.location.lng']
            print(1)
        except:
            continue
    return new_data

In [20]:
get_places(delhi_dataframe)

,Name,Categories,Latitude,Longitude


In [22]:
a=[]
new_data=pd.DataFrame(columns=['Name','Categories','Latitude','Longitude'])
row=top_delhi.iloc[3]
url_foursquare='https://api.foursquare.com/v2/venues/explore?&client_id={0[0]}&client_secret={0[1]}&v={0[2]}&ll={1},{2}&radius={0[4]}&limit={0[3]}'.format(get_credentials(),row['Latitude'],row['Longitude'])
result=requests.get(url_foursquare).json()
data=json_normalize(result['response']['groups'][0]['items'])
new_data.iloc[index,'Name']=data['venue.name']
new_data.iloc[index,'Categories']=data['venue.categories']
new_data.iloc[index,'Latitude']=data['venue.location.lat']
new_data.iloc[index,'Longitude']=data['venue.location.lng']
new_data
red

KeyError: 'groups'

In [23]:
result

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '60910e709472455a0168a2c9'},
 'response': {}}

In [ ]:
a